# Testing
### Mietpreise in der Stadt Zürich 2022

## Beschreibung
Der Datensatz enthält die Resultate der Mietpreiserhebung 2022. Enthalten sind die geschätzten Mietpreisbandbreiten für vier Raumstufen (Ganze Stadt, Stadtkreise, Stadtquartiere und Quartiergruppen). Siehe dazu unten unter «Bemerkungen» die weiteren Erläuterungen. Erleben Sie die Daten auch auf dem [MPE-Tool](https://www.stadt-zuerich.ch/content/prd/de/index/statistik/publikationen-angebote/datenbanken-anwendungen/mietpreiserhebung.html) von Statistik Stadt Zürich.

https://data.stadt-zuerich.ch/dataset/bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161/

Datum: 02.09.2022


### Importiere die notwendigen Packages

In [1]:
#%pip install geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly leafmap

In [2]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

import datetime
import geopandas as gpd
import folium 
import plotly.express as px
import seaborn as sns
import leafmap

import requests
import io

import plotly.express as px


In [3]:
SSL_VERIFY = False
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

In [4]:
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()

Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [5]:
#pd.options.display.float_format = lambda x : '{:,.1f}'.format(x) if (np.isnan(x) | np.isinf(x)) else '{:,.0f}'.format(x) if int(x) == x else '{:,.1f}'.format(x)
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import datetime und from datedtime import datetime, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [6]:
#today_date = datetime.date.today()
#date_time = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M')
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")

date_day_a_week_ago = (datetime.datetime.now() - datetime.timedelta(days=7)).date()
day_a_week_ago = date_day_a_week_ago.strftime('%Y-%m-%d')


print(now," vor einer Woche: ", day_a_week_ago)

2022-11-08  vor einer Woche:  2022-11-01


Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [7]:
#now = datetime.now() 
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
    'datenstand: ', selectedMonat,
     int_times)


2022 11 datenstand:  9 time.struct_time(tm_year=2022, tm_mon=11, tm_mday=8, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=312, tm_isdst=-1)


Berechne die Variable Epoche um später das SAS-Datum in ein Unix-Datum umzuwandeln. Bei SAS beginnt die Epoche am 1.1.1960. Bei Unix am 1.1.1970.
Diese Variable wird beim CSV-Import benötigt.

In [8]:
epoch = datetime.datetime(1960, 1, 1)

### Setze einige Pfadvariabeln

- Der Packagename ist eigentlich der **Verzeichnisname** unter dem die Daten und Metadaten auf der Dropzone abgelegt werden.
- Definiert wird er bei SASA-Prozessen auf dem **Produkte-Sharepoint ([Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/PersonalViews.aspx?PageView=Personal&ShowWebPart={6087A3E7-8AC8-40BA-8278-DECFACE124FF}))**.
- Der Packagename wird auf CKAN teil der URL, daher ist die exakte Schreibweise wichtig.

Beachte: im Packagename müssen alle Buchstaben **klein** geschrieben werden. Dies weil CKAN aus grossen kleine Buchstaben macht.

**BITTE HIER ANPASSEN**

In [9]:
package_name = "bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161"

In [10]:
dataset_name = "BAU516OD5161.csv"

**Statische Pfade in DWH-Dropzones**

In [11]:
dropzone_path_integ = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH"

In [12]:
dropzone_path_prod = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH"

**Statische Pfade CKAN-URLs**

In [13]:
ckan_integ_url ="https://data.integ.stadt-zuerich.ch/dataset/int_dwh_"

In [14]:
ckan_prod_url ="https://data.stadt-zuerich.ch/dataset/"

### Checke die Metadaten auf der CKAN INTEG- oder PROD-Webseite

Offenbar lassen sich aktuell im Markdownteil keine Variabeln ausführen, daher gehen wir wie unten gezeigt vor. Siehe dazu: https://data-dive.com/jupyterlab-markdown-cells-include-variables
Instead of setting the cell to Markdown, create Markdown from withnin a code cell! We can just use python variable replacement syntax to make the text dynamic

In [15]:
from IPython.display import Markdown as md

In [16]:
md(" **1. Dataset auf INTEG-Datakatalog:** Link {} ".format(ckan_integ_url+package_name))

 **1. Dataset auf INTEG-Datakatalog:** Link https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161 

In [17]:
md(" **2. Dataset auf PROD-Datakatalog:** Link {} ".format(ckan_prod_url+package_name))

 **2. Dataset auf PROD-Datakatalog:** Link https://data.stadt-zuerich.ch/dataset/bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161 

### Importiere einen Datensatz 

Definiere zuerst folgende Werte:
1) Kommt der Datensatz von PROD oder INTEG?
2) Beziehst Du den Datensatz direkt ab der DROPZONE oder aus dem INTERNET?

In [18]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "prod"; #prod vs something else
data_source = "web"; #dropzone vs something else
print(status+" - "+ data_source)

prod - web


In [19]:
# Filepath
if status == "prod":
    if data_source == "dropzone":
            fp = dropzone_path_prod+"\\"+ package_name +"\\"+dataset_name
            print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_prod_url+package_name+'/download/'+dataset_name
        print("fp lautet:"+fp)
else:
    if data_source == "dropzone":
        fp = dropzone_path_integ+"\\"+ package_name +"\\"+dataset_name
        print("fp lautet:"+fp)
    else:
        #fp = r"https://data.stadt-zuerich.ch/dataset/bau_neubau_whg_bausm_rinh_geb_projstatus_quartier_seit2009_od5011/download/BAU501OD5011.csv"
        fp = ckan_integ_url+package_name+'/download/'+dataset_name
        #fp = r"https://gist.githubusercontent.com/DonGoginho/de97fea04b0f229edf8e241b8207301d/raw/346f9dc06fc953ce7cd4d7de7cf2e823af21bf46/OGD_Export.csv"
        print("fp lautet:"+fp)


fp lautet:https://data.stadt-zuerich.ch/dataset/bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161/download/BAU516OD5161.csv


Beachte, wie das SAS Datum (ohne Format) in ein UNIX Datum umgerechnet und als Datumsformat dargestellt wird! Siehe dazu `https://stackoverflow.com/questions/26923564/convert-sas-numeric-to-python-datetime`

In [20]:
# Read the data

if data_source == "dropzone":
    data2betested = pd.read_csv(
        fp
        , sep=','
        ,parse_dates=['StichtagDatJahr', 'StichtagDatMonat']
        ,low_memory=False
    )
    print("dropzone")
else:
    r = requests.get(fp, verify=False)  
    r.encoding = 'utf-8'
    data2betested = pd.read_csv(
        io.StringIO(r.text)
        ,parse_dates=['StichtagDatJahr', 'StichtagDatMonat']
        # KONVERTIERE DAS SAS DATUM IN EIN UNIXDATUM UND FORMATIERE ES
        #, date_parser=lambda s: epoch + datetime.timedelta(days=int(s))
        , sep=','        
        ,low_memory=False)
    print("web")

data2betested.dtypes

web


StichtagDatJahr      datetime64[ns]
StichtagDatMonat     datetime64[ns]
RaumeinheitSort               int64
RaumeinheitLang              object
GliederungSort                int64
GliederungLang               object
ZimmerSort                    int64
ZimmerLang                   object
GemeinnuetzigSort             int64
GemeinnuetzigLang            object
EinheitSort                   int64
EinheitLang                  object
PreisartSort                  int64
PreisartLang                 object
mean                        float64
meanl                       float64
meanu                       float64
qu10                        float64
qu10l                       float64
qu10u                       float64
qu25                        float64
qu25l                       float64
qu25u                       float64
qu50                        float64
qu50l                       float64
qu50u                       float64
qu75                        float64
qu75l                       

Berechne weitere Attribute falls notwendig

In der Folge ein paar erste Tests:

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [21]:
data2betested.head(2)

,StichtagDatJahr,StichtagDatMonat,RaumeinheitSort,RaumeinheitLang,GliederungSort,GliederungLang,ZimmerSort,...,qu75u,qu90,qu90l,qu90u,Domain,Sample1,Sample2
0,2022-01-01,2022-04-01,1,Ganze Stadt,0,Ganze Stadt,2,...,1750,2155,2120,2200,46692,13488,1714
1,2022-01-01,2022-04-01,1,Ganze Stadt,1,Neubau bis 2 Jahre,2,...,2380,2490,2273,2914,1498,488,60


In [22]:
#data2betested.dtypes

In [23]:
data2betested.shape

(1896, 35)

Beschreibe einzelne Attribute

In [24]:
data2betested.describe()

,RaumeinheitSort,GliederungSort,ZimmerSort,GemeinnuetzigSort,EinheitSort,PreisartSort,mean,...,qu75u,qu90,qu90l,qu90u,Domain,Sample1,Sample2
count,1896,1896,1896,1896,1896,1896,1896,...,1896,1896,1896,1896,1896,1896,1896
mean,3,59,3,1,2,2,880,...,1126,1279,1184,1411,4778,1496,157
std,1,41,1,1,1,1,935,...,1222,1378,1267,1551,10274,3167,320
min,1,0,2,0,1,1,13,...,15,16,15,17,50,17,0
25%,2,20,2,0,1,1,23,...,28,32,29,35,927,274,38
50%,3,60,3,1,2,2,421,...,505,558,522,593,1846,586,64
75%,3,92,4,2,2,2,1660,...,2100,2404,2240,2633,3601,1174,106
max,4,129,4,2,2,2,3682,...,5871,5439,4471,7389,74180,23356,1908


Wie viele Nullwerte gibt es im Datensatz?

In [25]:
data2betested.isnull().sum()

StichtagDatJahr      0
StichtagDatMonat     0
RaumeinheitSort      0
RaumeinheitLang      0
GliederungSort       0
GliederungLang       0
ZimmerSort           0
ZimmerLang           0
GemeinnuetzigSort    0
GemeinnuetzigLang    0
EinheitSort          0
EinheitLang          0
PreisartSort         0
PreisartLang         0
mean                 0
meanl                0
meanu                0
qu10                 0
qu10l                0
qu10u                0
qu25                 0
qu25l                0
qu25u                0
qu50                 0
qu50l                0
qu50u                0
qu75                 0
qu75l                0
qu75u                0
qu90                 0
qu90l                0
qu90u                0
Domain               0
Sample1              0
Sample2              0
dtype: int64

### Verwende das Datum als Index

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. 
**To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame**

In [26]:
data2betested = data2betested.set_index("StichtagDatMonat")

In [27]:
#lambda x: x.StichtagDatMonat.astype(str)

In [28]:
#data2betested.info()
data2betested.index.unique()

DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='StichtagDatMonat', freq=None)

### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

In [29]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

### Zeitpunkte und Zeiträume abfragen

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

In [30]:
#data2betested.loc["2021"].head(5)
#data2betested.loc["2021-10-31":"2021-11-30"].head(2)

### Visualisierungen nach Zeitausschnitten

 #### reminder: 
 Encoding Data Types https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types

#### Übersicht der Dimensionen

In [31]:
dim_raumeinheiten = data2betested\
    .groupby(['RaumeinheitSort', 'RaumeinheitLang']) \
    .agg(count=('RaumeinheitSort', 'count'),mean_qu50=('qu50', 'mean')) \
    .sort_values('RaumeinheitSort', ascending=True) 

dim_raumeinheiten

,,count,mean_qu50
RaumeinheitSort,RaumeinheitLang,,
1,Ganze Stadt,216,822
2,Stadtkreise,468,822
3,Quartiergruppen,792,808
4,Quartiere,420,922


In [32]:
dim_gliederung = data2betested\
    .query('GliederungSort !=0')\
    .groupby(['GliederungSort', 'GliederungLang']) \
    .agg(count=('GliederungSort', 'count'),mean_qu50=('qu50', 'mean')) \
    .sort_values('GliederungSort', ascending=True) 

#dim_gliederung.head(60)
dim_gliederung.head(30)

,,count,mean_qu50
GliederungSort,GliederungLang,,
1,Neubau bis 2 Jahre,36,1144
2,Neubezug bis 2 Jahre,36,856
3,Bestand Mietverträge 2–10 Jahre,36,826
4,Bestand Mietverträge 11-20 Jahre,36,707
5,Bestand Mietverträge über 20 Jahre,36,618
10,Kreis 1,72,1057
11,Rathaus,12,1147
12,Hochschulen,12,1279
13,Lindenhof,12,1325


In [33]:
dim_zimmer = data2betested\
    .query('GliederungSort !=0')\
    .groupby(['ZimmerSort','ZimmerLang']) \
    .agg(count=('ZimmerSort', 'count'),mean_qu50=('qu50', 'mean')) \
    .sort_values('ZimmerSort', ascending=True) 

#dim_zimmer.head(60)
dim_zimmer.head(4)

,,count,mean_qu50
ZimmerSort,ZimmerLang,,
2,2 Zimmer,592,708
3,3 Zimmer,592,824
4,4 Zimmer,592,994


In [34]:
dim_gemeinnuetzig = data2betested\
    .query('GliederungSort ==0')\
    .groupby(['GemeinnuetzigSort', 'GemeinnuetzigLang',]) \
    .agg(count=('GemeinnuetzigSort', 'count'),mean_qu50=('qu50', 'mean')) \
    .sort_values('GemeinnuetzigSort', ascending=True) 

#dim_gemeinnuetzig.head(60)
dim_gemeinnuetzig.head()

,,count,mean_qu50
GemeinnuetzigSort,GemeinnuetzigLang,,
0,Alle Wohnungen,48,823
1,Gemeinnützig,36,566
2,Nicht gemeinnützig,36,955


In [35]:
dim_Einheit = data2betested\
    .query('GliederungSort != 0')\
    .groupby(['EinheitSort', 'EinheitLang']) \
    .agg(count=('EinheitSort', 'count'),mean_qu50=('qu50', 'mean')) \
    .sort_values('EinheitSort', ascending=True) 

#dim_Einheit.head(60)
dim_Einheit.head(4)

,,count,mean_qu50
EinheitSort,EinheitLang,,
1,Wohnung,888,1662
2,Quadratmeter,888,22


In [36]:
dim_Preisart = data2betested\
    .query('GliederungSort !=0')\
    .groupby(['PreisartSort', 'PreisartLang']) \
    .agg(count=('PreisartSort', 'count'),mean_qu50=('qu50', 'mean')) \
    .sort_values('PreisartSort', ascending=True) 

#dim_Preisart.head(60)
dim_Preisart.head(4)

,,count,mean_qu50
PreisartSort,PreisartLang,,
1,Netto,888,797
2,Brutto,888,887


In [37]:
data2betested.columns

Index(['StichtagDatJahr', 'RaumeinheitSort', 'RaumeinheitLang', 'GliederungSort', 'GliederungLang',
       'ZimmerSort', 'ZimmerLang', 'GemeinnuetzigSort', 'GemeinnuetzigLang', 'EinheitSort',
       'EinheitLang', 'PreisartSort', 'PreisartLang', 'mean', 'meanl', 'meanu', 'qu10', 'qu10l',
       'qu10u', 'qu25', 'qu25l', 'qu25u', 'qu50', 'qu50l', 'qu50u', 'qu75', 'qu75l', 'qu75u',
       'qu90', 'qu90l', 'qu90u', 'Domain', 'Sample1', 'Sample2'],
      dtype='object')

#### Meine Selektionen

In [38]:
mySel1 = data2betested\
    .query(
    'GliederungSort >0 and \
    GemeinnuetzigSort < 10 and \
    EinheitSort == 1 and\
    PreisartSort ==1'
    )\
    .sort_values('GliederungSort', ascending=True) 
    #.groupby(['PreisartSort', 'PreisartLang']) \
    #.agg(count=('PreisartSort', 'count'),mean_qu50=('qu50', 'mean')) \


#mySel1.head(60)
mySel1.head(10)
print(mySel1)

                 StichtagDatJahr  RaumeinheitSort  RaumeinheitLang  GliederungSort  \
StichtagDatMonat                                                                     
2022-04-01            2022-01-01                1      Ganze Stadt               1   
2022-04-01            2022-01-01                1      Ganze Stadt               1   
2022-04-01            2022-01-01                1      Ganze Stadt               1   
2022-04-01            2022-01-01                1      Ganze Stadt               1   
2022-04-01            2022-01-01                1      Ganze Stadt               1   
...                          ...              ...              ...             ...   
2022-04-01            2022-01-01                3  Quartiergruppen             129   
2022-04-01            2022-01-01                3  Quartiergruppen             129   
2022-04-01            2022-01-01                3  Quartiergruppen             129   
2022-04-01            2022-01-01                3  Qua

In [39]:
myTitle="Mietpreise gemeinnütziger Wohnungen nach Stadtkreisen und Zimmerzahl"
myQuery = 'RaumeinheitSort ==2 and GemeinnuetzigSort == 1'

highlight = alt.selection(type='single', on='mouseover',
                          fields=['ProjStatus'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySel1.query(myQuery), title=myTitle).encode(
        x=alt.X('GliederungSort:Q', axis=alt.Axis(title='Stadtkreis'), scale=alt.Scale(domain=[5, 125]))# , axis=alt.Axis(format='%', title='percentage')
        , y=alt.X('qu50:Q', axis=alt.Axis(title='Median-Preis'), scale=alt.Scale(domain=[0, 3500]))
        , color=alt.Color('ZimmerSort:N', legend=alt.Legend(title="Zimmerzahl", orient="right"))  
        ,tooltip=['GliederungSort', 'GliederungLang', 'ZimmerSort','ZimmerLang', 'GemeinnuetzigSort', 'GemeinnuetzigLang', 'EinheitSort', 'EinheitLang',
       'PreisartSort', 'PreisartLang',  'qu50', 'qu50l', 'qu50u']
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.5), alt.value(4))
).interactive()

lines + points

alt.LayerChart(...)

In [59]:
myTitle="Mietpreise nicht gemeinnütziger Wohnungen nach Stadtkreisen und Zimmerzahl"
myQuery = 'RaumeinheitSort ==2 and GemeinnuetzigSort == 2'

highlight = alt.selection(type='single', on='mouseover',
                          fields=['ProjStatus'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySel1.query(myQuery), title=myTitle).encode(
        x=alt.X('GliederungSort:Q', axis=alt.Axis(title='Stadtkreis'), scale=alt.Scale(domain=[5, 125]))# , axis=alt.Axis(format='%', title='percentage')
        , y=alt.X('qu50:Q', axis=alt.Axis(title='Median-Preis'), scale=alt.Scale(domain=[0, 3500]))
        , color=alt.Color('ZimmerSort:N', legend=alt.Legend(title="Zimmerzahl", orient="right"))  
        ,tooltip=['GliederungSort', 'GliederungLang', 'ZimmerSort','ZimmerLang', 'GemeinnuetzigSort', 'GemeinnuetzigLang', 'EinheitSort', 'EinheitLang',
       'PreisartSort', 'PreisartLang',  'qu50', 'qu50l', 'qu50u']
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

lines + points

alt.LayerChart(...)

In [60]:
myTitle="Mietpreise aller Wohnungen nach Stadtquartier und Zimmerzahl"

myQuery = 'RaumeinheitSort ==4 and GemeinnuetzigSort == 0'

highlight = alt.selection(type='single', on='mouseover',
                          fields=['ProjStatus'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySel1.query(myQuery), title=myTitle).encode(
        x=alt.X('GliederungSort:Q', axis=alt.Axis(title='Stadtkreis'), scale=alt.Scale(domain=[5, 125]))# , axis=alt.Axis(format='%', title='percentage')
        , y=alt.X('qu50:Q', axis=alt.Axis(title='Median-Preis'), scale=alt.Scale(domain=[0, 3500]))
        , color=alt.Color('ZimmerSort:N', legend=alt.Legend(title="Zimmerzahl", orient="right"))  
        ,tooltip=['GliederungSort', 'GliederungLang', 'ZimmerSort','ZimmerLang', 'GemeinnuetzigSort', 'GemeinnuetzigLang', 'EinheitSort', 'EinheitLang',
       'PreisartSort', 'PreisartLang',  'qu50', 'qu50l', 'qu50u']
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

lines + points

alt.LayerChart(...)

--------------- stopped

#### Entwicklung fertigerstellter Projekte nach Stadtkreis

In [ ]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAgg = data2betested.loc["1996":"2021"].query('ProjStatus == "Fertigerstellt"')\
    .groupby(['Jahr','KreisLang']) \
    .agg(sum_ProjAnzGbd=('ProjAnzGbd', 'sum'),sum_ProjAnzWhg=('ProjAnzWhg', 'sum')) \
    .sort_values('Jahr', ascending=True) 

myAgg.reset_index().head(3)

In [ ]:
myTitle="Entwicklung fertigerstellter Projekte nach Stadtkreis"

highlight = alt.selection(type='single', on='mouseover',
                          fields=['KreisLang'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(myAgg.reset_index(), title=myTitle).encode(
    x=alt.X('Jahr', axis=alt.Axis(title='Jahr'))# , axis=alt.Axis(format='%', title='percentage')
    , y=alt.X('sum_ProjAnzGbd', axis=alt.Axis(title='Anz. Projektierte Gebäude'))
    , color=alt.Color('KreisLang', legend=alt.Legend(title="Stadtkreis", orient="right"))  
    ,tooltip=['Jahr', 'KreisLang','sum_ProjAnzGbd', 'sum_ProjAnzWhg']    
)
points = base.mark_circle().encode(
    opacity=alt.value(0.75)
).add_selection(
    highlight
).properties(
    width=750 , height=350
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.5), alt.value(4))
).interactive()

lines + points

#### Barcharts mit Seaborn

In [ ]:
#myAgg = data2betested.loc["2008-11-30":"2021-10-31"]
myAggBar = data2betested.loc["2021"]\
    .groupby(['Jahr', 'KreisLang','KreisSort', 'ProjStatus']) \
    .agg(sum_ProjAnzGbd=('ProjAnzGbd', 'sum'),sum_ProjAnzWhg=('ProjAnzWhg', 'sum')) \
    .sort_values('KreisLang', ascending=True) 

myAggBar.reset_index().head(3)

In [ ]:
#help(sns.catplot)     

In [ ]:
#Stile for the following sns graphs
sns.set_theme(style="whitegrid")

In [ ]:
#sns.color_palette("flare", as_cmap=True)
myHist = sns.catplot(x="KreisLang"
            , y="sum_ProjAnzWhg"
            , hue="ProjStatus"
            , kind="bar" #boxen, violin, --> wenn die aggregation nicht genau den werten in der grafik entspricht
            , palette="pastel" #, color="green"
            , height=5
            , aspect=2
            , order=None, legend_out=True
            ,data=myAggBar.reset_index().sort_values('KreisSort', ascending=True) 
           )
myHist.set_xlabels('Stadtkreis', fontsize=11) # not set_label
myHist.set_ylabels('Anz. Wohnungen', fontsize=11)


#### Scatterplots mit Seaborn 

##### Entwicklung nach Stadtkreisen

In [ ]:
myAggScat = data2betested.loc["2000":"2021"]\
    .groupby(['Jahr', 'KreisLang', 'KreisSort','ProjStatus']) \
    .agg(sum_ProjAnzGbd=('ProjAnzGbd', 'sum'),sum_ProjAnzWhg=('ProjAnzWhg', 'sum')) \
    .sort_values('KreisSort', ascending=True) 

In [ ]:
#myAggScat.reset_index()

In [ ]:
g = sns.FacetGrid(myAggScat.reset_index(), col="KreisLang", hue="ProjStatus", col_wrap=4, height=4,)
#g.map(sns.scatterplot, "Jahr", "sum_ProjAnzWhg", alpha=.8)
g.map(sns.lineplot, "Jahr","sum_ProjAnzWhg",alpha=.8,label='label2')

#Layout
g.set_axis_labels("","Projektierte Whgen")
g.set_titles(col_template="{col_name}", row_template="{row_name}")
#g.set(xlim=(0, 60), ylim=(0, 12), xticks=[10, 30, 50], yticks=[2, 6, 10])
#g.tight_layout()

g.add_legend()

##### Entwicklung nach Stadtquartieren

In [ ]:
myAggScat = data2betested.loc["2000":"2021"]\
    .groupby(['Jahr', 'QuarLang', 'QuarSort','ProjStatus']) \
    .agg(sum_ProjAnzGbd=('ProjAnzGbd', 'sum'),sum_ProjAnzWhg=('ProjAnzWhg', 'sum')) \
    .sort_values('QuarSort', ascending=True) 

In [ ]:
#Link to Seaborn API Description:
# https://seaborn.pydata.org/generated/seaborn.FacetGrid.html#seaborn.FacetGrid

In [ ]:
g = sns.FacetGrid(myAggScat.reset_index(), col="QuarLang", hue="ProjStatus", col_wrap=6, height=3,)
#g.map(sns.scatterplot, "Jahr", "sum_ProjAnzWhg", alpha=.8)
g.map(sns.lineplot, "Jahr","sum_ProjAnzWhg",alpha=.8,label='label2')

#Layout
g.set_axis_labels("","Projektierte Whgen")
g.set_titles(col_template="{col_name}", row_template="{row_name}")
#g.set(xlim=(0, 60), ylim=(0, 12), xticks=[10, 30, 50], yticks=[2, 6, 10])
#g.tight_layout()

g.add_legend()

### Daten in interaktiver Treemap zeigen
Dazu gibt es eine sehr nützliche Webseite https://plotly.com/python/treemaps/
Zu Farbskalen, siehe: https://plotly.com/python/builtin-colorscales/

#### Anzahl Wohnungen in BAUPROJEKTEN nach Projektstatus, Kreis und Quartier 2021

In [ ]:
data2betested.dtypes

In [ ]:
myTreemapAgg = data2betested.loc["2021"]  \
    .groupby(['Jahr', 'QuarLang', 'KreisLang', 'ProjStatus']) \
    .agg(sum_ProjAnzGbd=('ProjAnzGbd', 'sum'),sum_ProjAnzWhg=('ProjAnzWhg', 'sum'))\
    .sort_values('sum_ProjAnzWhg', ascending=False) 

myTreemapAgg.reset_index().head(3)

In [ ]:
fig = px.treemap(myTreemapAgg.query('sum_ProjAnzWhg >0').reset_index(), path=[px.Constant("Anzahl Wohnungen nach Projektstatus, Kreis und Quartier"),'ProjStatus', 'KreisLang', 'QuarLang']
                 , values='sum_ProjAnzWhg'
                 , color='sum_ProjAnzWhg'
                 ## bei discrete scales (bei Klassen und Strings):                 
                 #, color_discrete_map={'(?)':'lightgrey','EVP':'gold', 'SP':'#FF3030', 'PdA':'#EE3B3B', 'Grüne':'#A2CD5A','GLP':'#CAFF70','Die Mitte':'orange','FDP':'#104E8B','AL':'deeppink','SVP':'forestgreen', 'FL ZÜRI':'#8B864E','Volt':'#lightblue' }                
                 ## bei continuous scales (bei Zahlenwerten):
                , color_continuous_scale='cividis'
                #, color_continuous_midpoint=np.average(df['AnzBestWir'], weights=df['AnzBestWir'])  
                 ,height=400
                 ,width=1100                 
                )
fig.update_traces(root_color="grey")
fig.update_layout(margin = dict(t=50, l=25, r=250, b=25))
fig.show()

#### Anzahl Wohnungen nach Projektstatus, Kreis und Quartier

In [ ]:
data2betested.dtypes

In [ ]:
myTreemapAgg = data2betested.loc["2021"]  \
    .groupby(['Jahr', 'QuarLang', 'KreisLang', 'ProjStatus']) \
    .agg(sum_ProjKosten=('ProjKosten', 'sum'), sum_Whg_1_3_Zi=('Whg_1_3_Zi', 'sum'),sum_Whg_4plus_Zi=('Whg_4plus_Zi', 'sum'))\
    .sort_values('sum_Whg_1_3_Zi', ascending=False) 

myTreemapAgg.reset_index().head(3)

In [ ]:
fig = px.treemap(myTreemapAgg.query('sum_ProjKosten >0').reset_index(), path=[px.Constant("Anzahl Wohnungen nach Projektstatus, Kreis und Quartier"),'ProjStatus', 'KreisLang', 'QuarLang']
                 , values='sum_ProjKosten'
                 , color='ProjStatus'
                 ## bei discrete scales (bei Klassen und Strings):                 
                 #, color_discrete_map={'(?)':'lightgrey','EVP':'gold', 'SP':'#FF3030', 'PdA':'#EE3B3B', 'Grüne':'#A2CD5A','GLP':'#CAFF70','Die Mitte':'orange','FDP':'#104E8B','AL':'deeppink','SVP':'forestgreen', 'FL ZÜRI':'#8B864E','Volt':'#lightblue' }                
                 ## bei continuous scales (bei Zahlenwerten):
                , color_continuous_scale='cividis'
                #, color_continuous_midpoint=np.average(df['AnzBestWir'], weights=df['AnzBestWir'])  
                 ,height=400
                 ,width=1100                 
                )
fig.update_traces(root_color="grey")
fig.update_layout(margin = dict(t=50, l=25, r=250, b=25))
fig.show()

In [ ]:
#data2betested.dtypes

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](https://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/DispForm.aspx?ID=140&ContentTypeId=0x0100988EAF029F1EFE4CA675F53C32A5D53D01006DBC563E6FBE9E4EB6FDC780799752E1)**

## ---------------------- hier Plausi beendet

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

### Test: Choroplethenkarte
Importiere die Geodaten als GeoJSON

In [ ]:
# Read abt. Geopandas https://geopandas.org/docs/user_guide/io.html
# Wenn die Daten lokal agelegt sind, dann kannst Du folgendes machen:
##input_stadtquartiere = r"//szh.loc/ssz/data/GIS/Daten/Vektor/_aktuell/stzh.adm_statistische_quartiere_map.json"
##df_stadtquartiere = gpd.read_file(input_stadtquartiere)
##df_stadtquartiere.head(2)
#df_stadtkreise.dtypes

geojson_url = "https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Statistische_Quartiere?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=adm_statistische_quartiere_map"
df_adm_statistische_quartiere_map = gpd.read_file(geojson_url)

df_adm_statistische_quartiere_map.head(5)


Joine die importierten statistischen Daten des aktuellen Jahres zum Geodatensatz:

*Siehe dazu Doku zu [Geopandas](https://geopandas.org/docs/user_guide/mergingdata.html)*

In [ ]:
akt_wbev_quart = data2betested.loc["2021-11"].groupby(
    ['StichtagDatJahr','StichtagDatMM','QuarCd','QuarLang']
).agg(
    {'AnzBestWir':'sum'}
)
akt_wbev_quart.head()


In [ ]:
# Rename attribute stznr to StatZoneSort, so the IDs have the same name
stadtquartiere_shapes = df_adm_statistische_quartiere_map[['geometry', 'qnr', 'qname', 'knr']].rename(columns={'qnr': 'QuarCd'})

# Merge with `merge` method on shared variable (stznr und StatZoneSort):
stadtquartiere_shapes_joined = stadtquartiere_shapes.merge(akt_wbev_quart, on='QuarCd')

stadtquartiere_shapes_joined.head(2)


In [ ]:
# Plot by StatQuartiere
# Weitere Schemes: scheme='quantiles', scheme='fisherjenks', scheme='natural_breaks',

stadtquartiere_shapes_joined.plot(column='AnzBestWir', cmap='viridis', scheme='natural_breaks', legend=True )
#scheme werte: https://matplotlib.org/2.0.2/users/colormaps.html

stadtquartiere_shapes_joined.plot(column='AnzBestWir', cmap='plasma', scheme='fisherjenks', legend=True )
#scheme werte: https://matplotlib.org/2.0.2/users/colormaps.html
stadtquartiere_shapes_joined.plot(column='AnzBestWir', cmap='cool', scheme='quantiles', legend=True )
#scheme werte: https://matplotlib.org/2.0.2/users/colormaps.html

#
#Neuere Gallerie: https://matplotlib.org/stable/gallery/index.html

Noch zu prüfen, folgendes schöne Beispiel: https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html

Konkrete Beispiele von Bookeh in Jupyter NB: https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html